In [1]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import Counter
from gensim.test.utils import datapath
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import export_text
import graphviz 

In [2]:
def retrieve_feature_label(path):
    df = pd.read_pickle(path)
    return df['total_notes'], df['label']
X, Y = retrieve_feature_label("../../Youth_Recidivism_Project/Dataset/train.pickle")
lda_model = gensim.models.ldamodel.LdaModel.load('../../Youth_Recidivism_Project/LDA_Output/lda.model')
with open("../../Youth_Recidivism_Project/LDA_Output/corpus.txt", "rb") as fp:   # Unpickling
    corpus = pickle.load(fp)
predicted_labels = lda_model.get_document_topics(corpus)

In [3]:
new_label = []
prob = []
for i in predicted_labels:
    label,probability = max(i, key=lambda x:x[1])
    new_label.append(label)
    prob.append(probability)
new_label = np.array(new_label)

In [4]:
z = [0] * len(new_label)
pd_new = pd.DataFrame({'new':new_label,"z":z})
pd_new.columns = ['new','num']
pd_new_stat = pd_new.groupby("new").count()
print(pd_new_stat)

       num
new       
0    11398
1     3754
2    78909
3    12833
4      471
5    38952


In [5]:
file = pd.read_pickle("../../Youth_Recidivism_Project/Dataset/train.pickle")
for i in range(file.shape[0]):
    if file.iloc[i,19] != file.iloc[i,19]:
        file.iloc[i,19] = file.iloc[i,16]
file_stat = file[['WHO_WAS_CONTACTED','STAFF','WHY']].groupby(["STAFF"]).indices

In [6]:
print(file.iloc[0,:])

Unnamed: 0                                                                   0
MID                                                                     346868
FirstCommit_2                                                        2009/12/1
DischargeDate_2                                                       2016/7/5
CommitDays                                                                2408
monthsofstay                                                           79.2105
cnpermnth                                                              0.16412
numcasenote                                                                 13
numcaseworkers                                                               1
numcasenoteCW                                                               13
Treatment                                                            Treatment
YES                                                                          0
YESStartDate_2                                      

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

MID = file['MID'].to_numpy()
MID = MID.astype(str)
MID = le.fit_transform(MID)

YES = file['YES'].to_numpy()
YES = le.fit_transform(YES)

CaseworkeratDischargeDate = file['CaseworkeratDischargeDate'].to_numpy()
CaseworkeratDischargeDate[CaseworkeratDischargeDate != CaseworkeratDischargeDate] = ""
CaseworkeratDischargeDate = le.fit_transform(CaseworkeratDischargeDate)

CONTACT_TYPE = file['CONTACT_TYPE'].to_numpy()
CONTACT_TYPE[CONTACT_TYPE!=CONTACT_TYPE] = ""
CONTACT_TYPE = le.fit_transform(CONTACT_TYPE)

Staffing = file['Staffing'].to_numpy()
Staffing[Staffing!=Staffing] = ""
Staffing = le.fit_transform(Staffing)

#HOW_WAS_CONTACT_MADE = file['HOW_WAS_CONTACT_MADE'].to_numpy()
#HOW_WAS_CONTACT_MADE[HOW_WAS_CONTACT_MADE !=HOW_WAS_CONTACT_MADE] = ""
#HOW_WAS_CONTACT_MADE = le.fit_transform(HOW_WAS_CONTACT_MADE)

WHY = file['WHY'].to_numpy()
WHY[WHY!=WHY] = ""
#WHY = le.fit_transform(WHY)

STAFF = file['STAFF'].to_numpy()
STAFF[STAFF!=STAFF] = ""
STAFF = le.fit_transform(STAFF)

CommitDays = file['CommitDays'].to_numpy()
CommitDays[CommitDays!=CommitDays] = 0

numcasenote = file['numcasenote'].to_numpy()
numcasenote[numcasenote != numcasenote] = 0

numcaseworkers = file['numcaseworkers'].to_numpy()
numcaseworkers[numcaseworkers != numcaseworkers] = 0

numcasenoteCW = file['numcasenoteCW'].to_numpy()
numcasenoteCW[numcasenoteCW != numcasenoteCW] = 0

FirstCommit_2 = file['FirstCommit_2'].to_numpy()
FirstCommit_2[FirstCommit_2!=FirstCommit_2] = ""
FirstCommit_2 = FirstCommit_2.astype(str)
FirstCommit_2 = le.fit_transform(FirstCommit_2)

DischargeDate_2 = file['DischargeDate_2'].to_numpy()
DischargeDate_2[DischargeDate_2!=DischargeDate_2] = ""
DischargeDate_2 = DischargeDate_2.astype(str)
DischargeDate_2 = le.fit_transform(DischargeDate_2)


WHO_WAS_CONTACTED = file['WHO_WAS_CONTACTED'].to_numpy()
WHO_WAS_CONTACTED[WHO_WAS_CONTACTED!=WHO_WAS_CONTACTED] = ""
WHO_WAS_CONTACTED = WHO_WAS_CONTACTED.astype(str)
#WHO_WAS_CONTACTED = le.fit_transform(WHO_WAS_CONTACTED)
new_data = pd.DataFrame({"YES":YES
                        })

In [8]:
def one_hot_encoder(col_name):
    WHY = file[col_name].to_numpy()
    WHY_data = []
    for i in WHY:
        if i == i:
            i = i.replace(" ", "").split(",")
            i.sort()
            WHY_data.append(i)
        else:
            WHY_data.append([""])
    WHY_SET = []
    
    for i in WHY:
        if i == i:
            i = i.replace(" ", "").split(",")
            i.sort()
            WHY_SET +=i
        else:
            WHY_SET.append("")
    WHY_SET = list(set(WHY_SET))
    WHY_SET.sort()
    
    WHY_column = []
    for i in range(len(WHY_data)):
        temp = [0 for i in range(len(WHY_SET))]
        for z in range(len(WHY_SET)):
            if WHY_SET[z] in WHY_data[i]:
                temp[z] = 1
        WHY_column.append(temp)
    WHY_column = np.array(WHY_column) 
    WHY_final = pd.DataFrame(WHY_column)
    WHY_final.columns = WHY_SET
    return WHY_final

WHY = one_hot_encoder("WHY")

In [9]:
def one_hot_encoder_for_WHO():
    column = []
    data1 = file['WHO_WAS_CONTACTED'].to_numpy()
    data = []
    for i in data1:
        i = i.replace(" ", "")
        data.append(i)
        
    SET = ['', 'Adultcorrections', 'Client', 'CommunityActivity', 'Court', 
 'DistrictOfficevisit', 'ElectronicCommunications', 'Employmentvisit'
 , 'ExternalCollateral', 
 'Family', 'FamilyMember(s)', 'FamilySupport(s)', 'Father', 
 'Guardian', 'HomeofOther', 'Homevisit', 
 'InternalCollateral', 'MedicalOffice/Clinic', 'Mother', 'Non-CWStaffing', 
 'OtherAgency', 'Parent/Guardian', 'Policestation', 
 'Programvisit', 'Schoolvisit', 'Staffing(CWuseonly)', 'Streetcontact',
 'TransportOnly']
    final = []
    for i in range(len(data)):
        temp = [0 for i in range(len(SET))]
        for j in range(len(SET)):
            if SET[j] in data[i]:
                temp[j] = 1
        final.append(temp)
    final = np.array(final)
    final = pd.DataFrame(final)
    final.columns = SET
    final = final.drop(columns = [""])
    return final
WHO_WAS_CONTACTED = one_hot_encoder_for_WHO()

In [10]:
print(WHY.iloc[0,:])

                          0
AncillaryServices         0
BehaviorManagement        0
Check-in                  1
ClothingVoucher           0
CommunityService          0
CrisisScreening           0
CurfewManagement          0
Dental                    0
DrugScreening             0
Educational               0
Educational/Vocational    0
FamilyRelationships       0
HealthServices            0
Housing                   0
InitialVisit              0
Legal                     0
LevelReview               0
LifeSkills                0
Meeting                   0
Pass                      0
Psychiatric               0
Psychological             0
RRT                       0
Recreation                0
RevocationHearing         0
SocialServices            0
SubstanceAbuse            0
SupervisedVisit           0
Vocational                0
Name: 0, dtype: int64


In [11]:
data = pd.concat([WHY, WHO_WAS_CONTACTED], axis=1, join='inner')
data["CommitDays"] = CommitDays
data["CONTACT_TYPE"] = CONTACT_TYPE
data["YES"] = YES
data["MID"] = MID
data["CaseworkeratDischargeDate"] = CaseworkeratDischargeDate
data["Staffing"] = Staffing
data["STAFF"] = STAFF
data["CommitDays"] = CommitDays
data["numcasenote"] = numcasenote
data["numcaseworkers"] = numcaseworkers
data["numcasenoteCW"] = numcasenoteCW
data['FirstCommit_2'] = FirstCommit_2
data["DischargeData_2"] = DischargeDate_2
data["label"] = new_label

In [12]:
data.to_csv("../../Youth_Recidivism_Project/Dataset/full_DTdata.csv",index = False)